In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict
from IPython import display

In [3]:
def getdata():
    data_dir = '/content/dogscats'
    
    train_transforms = transforms.Compose([transforms.Resize([224,224]),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485,0.456,0.406],
                                                                [0.229,0.224,0.225])])
    
    valid_transforms = transforms.Compose([transforms.Resize([224,224]),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485,0.456,0.406],
                                                                [0.229,0.224,0.225])])
    
    train_data = datasets.ImageFolder(data_dir+ '/train', transform=train_transforms)
    valid_data = datasets.ImageFolder(data_dir+ '/valid', transform=valid_transforms)
    
    trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_data, batch_size=64)
    
    return [trainloader, validloader]

In [4]:
def freeze_model(model):
    for params in model.parameters():
        params.requires_grad = False
    return model

In [5]:
def model_train(model, trainloader, validloader, criterion, optimizer, epochs = 1, plot=False):
    steps = 0
    running_loss = 0
    running_losses = []
    valid_losses = []
    if not plot:
        print_every=5
    else:
        print_every=1
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device);
    for epoch in range(epochs):
        for inputs, labels in trainloader:
            steps += 1
            # Move input and label tensors to the default device
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            logps = model.forward(inputs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_losses.append(running_loss)
            if steps % print_every == 0:
                valid_loss = 0
                accuracy = 0
                model.eval()
                with torch.no_grad():
                    for inputs, labels in validloader:
                        inputs, labels = inputs.to(device), labels.to(device)
                        logps = model.forward(inputs)
                        batch_loss = criterion(logps, labels)
                        valid_loss += batch_loss.item()
                        # Calculate accuracy
                        ps = torch.exp(logps)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    valid_losses.append(valid_loss/(len(validloader)) )
                display.clear_output(wait=True)
                display.display(f"Epoch {epoch+1}/{epochs}.. "
                      f"Train loss: {running_loss/print_every:.3f}.. "
                      f"Valid loss: {valid_loss/len(validloader):.3f}.. "
                      f"Valid accuracy: {accuracy/len(validloader):.3f}")
                if (steps > 1) and plot:
                    plt.plot(running_losses, 'b', linewidth=2.0,
                             label="Training-Loss" if steps == 2 else "_nolegend_")
                    plt.plot(valid_losses, 'y', linewidth=2.0,
                             label="Validation-Loss" if steps == 2 else "_nolegend_")
                    if steps == 2:
                        plt.legend()
                    display.display(plt.gcf())
                running_loss = 0
                model.train()

In [6]:
dataloaders = getdata()

trainloader = dataloaders[0]
validloader = dataloaders[1]

In [7]:
model = models.resnet50(pretrained=True)

In [8]:
model = freeze_model(model)

In [9]:
classifer = nn.Sequential(OrderedDict([
                         ('fc1', nn.Linear(2048, 1024)),
                         ('relu', nn.ReLU()),
                         ('fc2', nn.Linear(1024, 512)),
                         ('relu', nn.ReLU()),
                         ('fc3', nn.Linear(512,2)),
                         ('output', nn.LogSoftmax(dim=1))]))

In [10]:
model.fc = classifer

In [11]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

In [12]:
model_train(model, trainloader, validloader, criterion, optimizer )

'Epoch 1/1.. Train loss: 0.040.. Valid loss: 0.046.. Valid accuracy: 0.982'